In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Gezgin Satıcı Probleminin Basit Genetik Algoritma İle Çözülmesi (Populer Travelling Santa Örnek Verisi ile Dumbest(Aptal yaklaşımlarla ilerleyerek Sonuca Ulaşılmıştır)

## Paketlerin ve Kütüphanelerin İndirilmesi

In [ ]:
pip install six

In [ ]:
pip install mlrose

In [ ]:
#sklearn.externals.six paketinin hatasının çözümü maksadıyla ayrıca modül indirimi yapılmıştır.
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
import pandas as pd
from geopy import distance
import requests # to call the openmap/google apis
import json
import datetime
import math
import itertools

# Eğer sklearn.external six paketi ile problemle karşılaşır iseniz yardımcı link: https://stackoverflow.com/questions/61867945/python-import-error-cannot-import-name-six-from-sklearn-externals
# Normalde original paket indirimi şöyle yapılır:
#from sklearn.external import six
# to:
# import six
import mlrose #travelling salesman problem çözümü için

# that was fixed in mlrose_hiive, though the outputs are different
#import mlrose_hiive
import numpy as np
import six
import sys
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
import pandas as pd
from geopy import distance
import six
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import random
import numpy as np

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import math
import pandas as pd
from sympy import isprime
from multiprocessing import Pool


In [ ]:
df_cities= pd.read_csv('../input/traveling-santa-2018-prime-paths/cities.csv')
df_cities.head()

 Çalışmaya, NP-Complete problemini daha iyi anlayabilmek amacıyla ilk önce bazı basit aptal yaklaşımlarla başlanılmıştır  ve hangi aşamaya götürdükleri test edilmiştir.

Tüm şehirlerin dağılım grafiğinin çıktısı alıyoruz.

In [ ]:
fig = plt.figure(figsize=(10,10))
#cmap, norm = from_levels_and_colors([0.0, 0.5, 1.5], ['red', 'black'])
plt.scatter(df_cities['X'],df_cities['Y'],marker = '.',
            c=(df_cities.CityId != 0).astype(int), cmap='Set1', 
            alpha = 0.6, s = 500*(df_cities.CityId == 0).astype(int)+1)
plt.show()

Şehirler bir ren geyiği düzeninde düzenlenmiş gibi görünüyor.

Kırmızı nokta Kuzey Kutbu'nu gösterir (CityId = 0).

Tek yapmamız gereken, kırmızı noktadan giden, diğer tüm noktalara dokunan ve kat edilen minimum toplam mesafe ile kırmızı noktaya geri dönen bir yol bulmaktır.

#### Dumbest Path-Dumbest güzergah(En Saçma-aptal Yol): CityID'lerin sırasına göre gidin: 0, 1, 2 .. vs. ve sona ulaştığınızda sıfıra geri dönün.

Soruna göre: "gönderim, gönderdiğiniz yolun Öklid mesafesine göre puanlanır, birincil CityId'den gelmedikçe her 10. adımın %10 daha uzun olması kısıtlamasına tabidir."

1-Bu yolun ne kadar iyi olduğunu görmek için birkaç işleve ihtiyacımız var:

2-Bir dizi sayı verilen toplam mesafeyi hesaplamak için başka bir işlev

In [ ]:
# The function to get the distance between the cities.
def distance(x1, y1, x2, y2, prev_is_prime, is_10th):
    # Every 10th step is 10% more lengthy unless coming from a prime CityId.
    cost_factor = 1.1 if is_10th and not prev_is_prime else 1.0
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * cost_factor

# The function to calculate score.

def calculate_score(path, cities_df_dict):
    # The beginning and end of the paths must be City'0'.
    paths = [0]
    paths.extend(path)
    paths.extend([0])

    sum_distance = 0
    prev_x, prev_y = cities_df_dict['X'][0], cities_df_dict['Y'][0]
    prev_is_prime = False

    for i, city in enumerate(paths):
        x, y = cities_df_dict['X'][city], cities_df_dict['Y'][city]
        is_prime = cities_df_dict['IsPrime'][city]

        sum_distance += distance(prev_x, prev_y, x, y, prev_is_prime, i % 10 == 0)
        prev_x, prev_y = x, y
        prev_is_prime = is_prime

    return sum_distance

In [ ]:
df_cities['IsPrime'] = df_cities.CityId.apply(isprime)
cities_df_dict       = df_cities.to_dict()

df_cities.head(4)

In [ ]:
%%time
# test time in calc
sample_path = df_cities.CityId[:].values[1:]
for i in range(100):
    score = calculate_score(sample_path, cities_df_dict)

#### Dumbest Path(En Aptal Yol)

In [ ]:
# start and end in the nort (0)
path  = df_cities.CityId[:].values[1:]
score = calculate_score(path, cities_df_dict)
print('Total distance with the dumbest path is '+ "{:,}".format(score))
#446884407.52

Dumbest pat'in (En aptal yolun) ilk 100 adımına bir göz atalım

In [ ]:
dumbest_path = path

def plot_sample_path(path):
    df_path = pd.merge_ordered(pd.DataFrame({'CityId':path}),df_cities,on=['CityId'])
    fig, ax = plt.subplots(figsize=(10,10))
    ax.plot(df_path.iloc[0:100,]['X'], df_path.iloc[0:100,]['Y'],marker = 'o')
    for i, txt in enumerate(df_path.iloc[0:100,]['CityId']):
        ax.annotate(txt, (df_path.iloc[0:100,]['X'][i], df_path.iloc[0:100,]['Y'][i]),size = 15)
        
plot_sample_path(dumbest_path)

Gördüğümüz gibi, en aptal yol oldukça kötü görünüyor. Satıcıyı veya Noel Baba'yı hiç düşünmeden haritanın her yerine göndermiş oluyoruz.

In [ ]:
df_cities.head(5)

In [ ]:
print("All cities: ", df_cities.shape)
print("Prime cities: ", df_cities[df_cities.IsPrime].shape)

### Genetik Algoritma(Genetic Algorithm)

Genetik algoritmalar, Darwin'in evrim teorisinden esinlenmiştir. Genetik algoritmalarla çözülen bir soruna çözüm geliştirilir.Algoritma, popülasyon adı verilen bir dizi çözümle (kromozomlarla temsil edilir) başlatılır. Bir popülasyondan çözümler alınır ve yeni bir popülasyon oluşturmak için kullanılır. Bu, yeni nüfusun eskisinden daha iyi olacağı umuduyla motive edilir. Yeni çözümler (yavrular) oluşturmak için seçilen çözümler uygunluklarına göre seçilir - ne kadar uygun olurlarsa üreme şansları o kadar artar.

[Başlat] Rastgele n kromozom popülasyonu oluşturun (soruna uygun çözümler).
​
[Uygunluk] Popülasyondaki her bir x kromozomunun uygunluğunu f (x) değerlendirin

[Yeni popülasyon] Yeni popülasyon tamamlanana kadar aşağıdaki adımları tekrarlayarak yeni bir popülasyon oluşturun

[Seçim] Bir popülasyondan uygunluklarına göre iki ebeveyn kromozomu seçin (daha iyi uygunluk, daha büyük seçilme şansı) 
​
[Çaprazlama] Çaprazlama olasılığı ile yeni bir yavru (çocuklar) oluşturmak için ebeveynleri çaprazlayın. Çaprazlama yapılmadıysa, yavrular ebeveynlerin tam bir kopyasıdır.

[Mutasyon] Bir mutasyon olasılığı ile her lokusta yeni yavruları mutasyona uğratın (kromozomdaki konum).

[Kabul] Yeni yavruları yeni bir popülasyona yerleştirin

[Değiştirin] Daha sonraki bir algoritma çalışması için yeni oluşturulan popülasyonu kullanın

[Test] Son koşul karşılanırsa, durdurun ve mevcut popülasyondaki en iyi çözümü geri verin

[Döngü] 2. adıma gidin.
[Döngü] 2. adıma gidin.

In [ ]:
"""Genetic Algorithmn Implementation
see:
http://www.obitko.com/tutorials/genetic-algorithms/ga-basic-description.php
"""
class GeneticAlgorithm(object):
    def __init__(self, genetics):
        self.genetics       = genetics
        self.best_solution  = None
        self.avg_fitness    = []
        pass

    def fitness(self, ch):
        return (self.genetics.fitness(ch),  ch)
    
    def run(self, processes=4):
        population = self.genetics.initial()
        pool       = Pool(processes=processes)
        
        while True:
            fits_pops = [self.fitness(ch) for ch in population]
            fits_pops = list(sorted(fits_pops))
            #fits_pops   = pool.map(self.fitness, population)

            # Log
            self.avg_fitness.append(np.mean([f for f, ch in fits_pops]))
            self.best_solution = fits_pops[0][1]
            
            if self.genetics.check_stop(fits_pops): 
                break
            
            population = self.next(fits_pops)
            pass
        return population

    def next(self, fits):
        parents_generator = self.genetics.parents(fits)
        size  = len(fits)
        nexts = [self.best_solution.copy()]
        while len(nexts) < size:
            parents  = next(parents_generator)
            cross    = random.random() < self.genetics.probability_crossover()
            children = self.genetics.crossover(parents) if cross else parents

            for ch in children:
                mutate = random.random() < self.genetics.probability_mutation()
                nexts.append(self.genetics.mutation(ch) if mutate else ch)
                pass
            pass
        return nexts[0:size]
    pass

In [ ]:
class TravelingSanta():
    def __init__(self, df_cities, generation=10, size=5,
                 prob_crossover=0.9, 
                 prob_mutation=0.2):
        self.df_cities  = df_cities
        
        self.counter   = 0
        
        self.generation = generation
        self.size  = size        
        self.prob_crossover = prob_crossover
        self.prob_mutation  = prob_mutation
        self.cities_df_dict = df_cities.to_dict()

    # GeneticFunctions interface impls
    def probability_crossover(self):
        return self.prob_crossover

    def probability_mutation(self):
        return self.prob_mutation

    # Initial population based in s0
    def initial(self):
        population = [self.random_chromo() for j in range(self.size)]
        return population

    def fitness(self, chromo):
        score = calculate_score(chromo, self.cities_df_dict)
        
        return score

    def check_stop(self, fits_populations):
        self.counter += 1
        
        if self.counter % 2 == 0:

            fits  = [f for f, ch in fits_populations]
            best  = min(fits)
            worst = max(fits)
            ave   = sum(fits) / len(fits)
            print(
                "[G %3d] score=(%2f, %2f, %2f, %2f): %r" %
                (self.counter, best, ave, worst, len(fits), []))
            pass

        return self.counter >= self.generation

    def parents(self, fits_populations):
        while True:
            father = self.tournament(fits_populations)
            mother = self.tournament(fits_populations)
            yield (father, mother)
            pass
        pass

    def crossover(self, parents):
        ind1, ind2 = parents
        
        size = min(len(ind1), len(ind2))
        a, b = random.sample(range(size), 2)
        if a > b:
            a, b = b, a

        holes1, holes2 = [True]*size, [True]*size
        for i in range(size):
            if i < a or i > b:
                holes1[ind2[i]-1] = False
                holes2[ind1[i]-1] = False

        # We must keep the original values somewhere before scrambling everything
        temp1, temp2 = ind1, ind2
        k1 , k2 = b + 1, b + 1
        for i in range(size):
            if not holes1[temp1[(i + b + 1) % size]-1]:
                ind1[k1 % size] = temp1[(i + b + 1) % size]
                k1 += 1

            if not holes2[temp2[(i + b + 1) % size]-1]:
                ind2[k2 % size] = temp2[(i + b + 1) % size]
                k2 += 1

        # Swap the content between a and b (included)
        for i in range(a, b + 1):
            ind1[i], ind2[i] = ind2[i], ind1[i]

        return ind1, ind2

    def mutation(self, chromosome):
        mutated = chromosome
        
        for i in range(int(len(chromosome)*0.05)):
            i1 = random.randint(0, len(chromosome)-1)
            i2 = random.randint(0, len(chromosome)-1)
        
            origin       = mutated[i1]
            mutated[i1]  = mutated[i2]
            mutated[i2]  = origin

        return mutated

    def tournament(self, fits_populations):
        alicef, alice = self.select_random(fits_populations)
        bobf, bob = self.select_random(fits_populations)
        return alice if alicef < bobf else bob

    def select_random(self, fits_populations):
        return fits_populations[random.randint(0, len(fits_populations)-1)]

    def random_chromo(self):
        chromo = list(df_cities.CityId.sample(frac=1).values)
        return chromo
    pass

 ### Genetik Algoritma Hesaplama

In [ ]:
%%time

generation      = 100
population_size = 100
prob_crossover  = 0.9
prob_mutation   = 0.2

ga = GeneticAlgorithm(
    TravelingSanta(df_cities, generation=generation, size=population_size, 
                   prob_crossover=prob_crossover, prob_mutation=prob_mutation)
)

best_solution = ga.run()

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(ga.avg_fitness)

### Çıktı 

In [ ]:
path       = [0] + ga.best_solution + [0]
submission = pd.DataFrame({"Path": path})
submission.to_csv("gezgin_satıcı_problemi_output_ga.csv", index=None)

### Gezgin Satıcı Probleminin En Yakın Komşuluk Algorithması ile /Greedy Çözüme Ulaştırılması---Genetik Algortima İle Aynı Veri Setine Sahip Çalışmadır.Populer Harita Gezgin Santa Harita Verileri Kullanılmıştır

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
from matplotlib import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import random 


In [ ]:
df_cities= pd.read_csv('../input/traveling-santa-2018-prime-paths/cities.csv')
df_cities.head()

In [ ]:
fig = plt.figure(figsize=(20,20))
#cmap, norm = from_levels_and_colors([0.0, 0.5, 1.5], ['red', 'black'])
plt.scatter(df_cities['X'],df_cities['Y'],marker = '.',c=(df_cities.CityId != 0).astype(int), cmap='Set1', alpha = 0.6, s = 500*(df_cities.CityId == 0).astype(int)+1)
plt.show()

Kırmızı nokta Kuzey Kutbu'nu gösterir (CityId = 0).Süreç, kırmızı noktadan giden, diğer tüm noktalara dokunan ve kat edilen minimum toplam mesafe ile kırmızı noktaya geri dönen bir yol bulma süreci olarak belirlenmiştir.

In [ ]:
# pozisyona göre asal olup olmadığı. 

# eratosten elek metodu kullanarak
def sieve_of_eratosthenes(n):
    primes = [True for i in range(n+1)] # Start assuming all numbers are primes
    primes[0] = False # 0 is not a prime
    primes[1] = False # 1 is not a prime
    for i in range(2,int(np.sqrt(n)) + 1):
        if primes[i]:
            k = 2
            while i*k <= n:
                primes[i*k] = False
                k += 1
    return(primes)
prime_cities = sieve_of_eratosthenes(max(df_cities.CityId))

In [ ]:
def total_distance(dfcity,path):
    prev_city = path[0]
    total_distance = 0
    step_num = 1
    for city_num in path[1:]:
        next_city = city_num
        total_distance = total_distance + \
            np.sqrt(pow((dfcity.X[city_num] - dfcity.X[prev_city]),2) + pow((dfcity.Y[city_num] - dfcity.Y[prev_city]),2)) * \
            (1+ 0.1*((step_num % 10 == 0)*int(not(prime_cities[prev_city]))))
        prev_city = next_city
        step_num = step_num + 1
    return total_distance

dumbest_path = list(df_cities.CityId[:].append(pd.Series([0])))
print('Total distance with the dumbest path is '+ "{:,}".format(total_distance(df_cities,dumbest_path)))

#### En aptal yolun ilk 100 adımına bir göz atalım.

In [ ]:
df_path = pd.merge_ordered(pd.DataFrame({'CityId':dumbest_path}),df_cities,on=['CityId'])
fig, ax = plt.subplots(figsize=(20,20))
ax.plot(df_path.iloc[0:100,]['X'], df_path.iloc[0:100,]['Y'],marker = 'o')
for i, txt in enumerate(df_path.iloc[0:100,]['CityId']):
    ax.annotate(txt, (df_path.iloc[0:100,]['X'][i], df_path.iloc[0:100,]['Y'][i]),size = 15)

#### Biraz daha iyi yol araştırması için: şehirler X,Y koordinatlarında sıralanmıştır.

In [ ]:
sorted_cities = list(df_cities.iloc[1:,].sort_values(['X','Y'])['CityId'])
sorted_cities = [0] + sorted_cities + [0]
print('Total distance with the sorted city path is '+ "{:,}".format(total_distance(df_cities,sorted_cities)))

#### Basit bir sıralama kullanarak toplam mesafemizi zaten yarıdan fazla azalttık !! Haritada nasıl göründüğüne bakalım (yakınlaştırılmış).

In [ ]:
df_path = pd.DataFrame({'CityId':sorted_cities}).merge(df_cities,how = 'left')
fig, ax = plt.subplots(figsize=(20,20))
ax.set_xlim(0,10)
ax.plot(df_path.iloc[0:100,]['X'], df_path.iloc[0:100,]['Y'],marker = 'o')
for i, txt in enumerate(df_path.iloc[0:100,]['CityId']):
    ax.annotate(txt, (df_path.iloc[0:100,]['X'][i], df_path.iloc[0:100,]['Y'][i]),size = 15)

#### Yukarıdaki çizimden, şimdi Satıcının(Noel Baba)'nın bir sonraki X'e geçmeden önce aynı X değeri (X ekseninin yalnızca 0-10 arasında değiştiğine dikkat edin) etrafında yukarı ve aşağı gittiğini görebiliriz. Bu biraz daha verimli bir yoldur.

#### Önceki Yolda görebildiğimiz sorunlardan biri, Satıcının(Noel Baba'nın) sistematik bir soldan sağa hareket etmesine rağmen, bunu yaparken Y ekseninin uç noktaları arasında hareket etmesidir.

#### Tüm haritayı X'ler ve Y'lerden oluşan bir ızgaraya bölersek, Satıcı(Noel Baba) bir sonraki kareye geçmeden önce ızgaradaki her kareyi kaplayabilir. Aşağıda gösterildiği gibi daha verimli olmalıdır.

In [ ]:
df_cities['Ycuts'] = pd.cut(df_cities.Y,300)
df_cities['Xcuts'] = pd.cut(df_cities.X,300)
grid_sorted_cities = list(df_cities.iloc[1:].sort_values(['Xcuts','Ycuts','X','Y'])['CityId'])
grid_sorted_cities =  [0] + grid_sorted_cities + [0]
print('Total distance with the sorted cities with a grid path is '+ "{:,}".format(total_distance(df_cities,grid_sorted_cities)))

In [ ]:
df_path = pd.DataFrame({'CityId':grid_sorted_cities}).merge(df_cities,how = 'left')
fig, ax = plt.subplots(figsize=(20,20))
ax.plot(df_path.iloc[0:1000,]['X'], df_path.iloc[0:1000,]['Y'],marker = 'o')

#### Puan açısından dilimlenmiş, sıralanmış şehirler ile büyük gelişme gerçekleşmiştir.

#### Yukarıdaki yolda görebileceğimiz yukarıdaki sorunlardan biri, Satıcının(Noel Baba'nın) en yüksek Y'ye sahip şehre ulaştığında, en düşük Y'ye sahip şehre geri dönmesi ve bunun aksine zikzak bir desen izleyerek yukarı doğru hareket etmeye başlamasıdır. hangisi daha verimli olmalıdır.

 #### Zig-zag yolu yönetmini  uygulayabiliyor muyuz ve biraz daha iyileştirme elde ediyor muyuz?

In [ ]:
zigzag_sorted_cities1 = list(df_cities.iloc[1:].sort_values(['Xcuts','Ycuts','X','Y'])['CityId'])
zigzag_sorted_cities2 = list(df_cities.iloc[1:].sort_values(['Xcuts','Ycuts','X','Y'], ascending = [True,False,True,True])['CityId'])
chooser_pattern = list(df_cities.iloc[1:].sort_values(['Xcuts']).groupby(['Xcuts']).ngroup()%2)

zigzag_cities = [zigzag_sorted_cities1[i] if chooser_pattern[i] == 0 else zigzag_sorted_cities2[i] for i in range(len(chooser_pattern))]
zigzag_cities =  [0] + zigzag_cities + [0]
print('Total distance with the Zig-Zag with grid city path is '+ "{:,}".format(total_distance(df_cities,zigzag_cities)))

Izgara düzeyinde bir zikzak yolu getirerek, Satıcı (Noel Baba) için en yüksek Y'den en düşük Y'ye olan uzun yolculukları ortadan kaldırdık ve puanı 3,2 milyondan 2,2 milyona daha da yükselttik.

In [ ]:
df_path = pd.DataFrame({'CityId':zigzag_cities}).merge(df_cities,how = 'left')
fig, ax = plt.subplots(figsize=(20,20))
ax.plot(df_path.iloc[0:1000,]['X'], df_path.iloc[0:1000,]['Y'],marker = 'o')

300x300 ızgara boyutunu keyfi olarak seçtik. Şimdi ızgara boyutunun bir fark yaratıp yaratmadığını görelim

In [ ]:
for i in range(100,600,100):
    for j in range(100,600,100):
        df_cities['Ycuts'] = pd.cut(df_cities.Y,j)
        df_cities['Xcuts'] = pd.cut(df_cities.X,i)
        zigzag_sorted_cities1 = list(df_cities.iloc[1:].sort_values(['Xcuts','Ycuts','X','Y'])['CityId'])
        zigzag_sorted_cities2 = list(df_cities.iloc[1:].sort_values(['Xcuts','Ycuts','X','Y'], ascending = [True,False,True,True])['CityId'])
        chooser_pattern = list(df_cities.iloc[1:].sort_values(['Xcuts']).groupby(['Xcuts']).ngroup()%2)

        zigzag_cities = [zigzag_sorted_cities1[i] if chooser_pattern[i] == 0 else zigzag_sorted_cities2[i] for i in range(len(chooser_pattern))]
        zigzag_cities =  [0] + zigzag_cities + [0]
        print('Total distance with the Zig-Zag with grid city path with grid size (' + str(i) + ',' + str(j) + ") is {:,}".format(total_distance(df_cities,zigzag_cities)))

In [ ]:
df_path = pd.DataFrame({'CityId':zigzag_cities}).merge(df_cities,how = 'left')
fig, ax = plt.subplots(figsize=(20,20))
ax.plot(df_path['X'], df_path['Y'])

(300.500), (300.300) 'den daha iyi ızgara boyutu gibi görünüyor, ama benim pek değil. Eh, tüm deneyler çalışmıyor.

#### Yukarıdaki algoritma ile son grafiğin nasıl göründüğüne bir göz atalım.

## ## En Yakın Komşu Algoritması/Greedy Sezgiseli

In [ ]:
def nearest_neighbour():
    
    cities= pd.read_csv('../input/traveling-santa-2018-prime-paths/cities.csv')
    ids = cities.CityId.values[1:]
    xy = np.array([cities.X.values, cities.Y.values]).T[1:]
    path = [0,]
    while len(ids) > 0:
        last_x, last_y = cities.X[path[-1]], cities.Y[path[-1]]
        dist = ((xy - np.array([last_x, last_y]))**2).sum(-1)
        nearest_index = dist.argmin()
        path.append(ids[nearest_index])
        ids = np.delete(ids, nearest_index, axis=0)
        xy = np.delete(xy, nearest_index, axis=0)
    path.append(0)
    return path

nnpath = nearest_neighbour()
print('Total distance with the Nearest Neighbor path '+  "is {:,}".format(total_distance(df_cities,nnpath)))

En yakın komşu algoritması, yaklaşık 2,2 milyon maliyetten 1,8 milyona önemli bir gelişme sağladı.

En yakın komşu algoritması ile yolun nasıl göründüğüne bakalım.

In [ ]:
df_path = pd.DataFrame({'CityId':nnpath}).merge(df_cities,how = 'left')
fig, ax = plt.subplots(figsize=(20,20))
ax.plot(df_path['X'], df_path['Y'])

## ### Prime Swaplı En Yakın Komşu / Açgözlü Algoritma(Greedy Algoritma)

Şimdiye kadar optimizasyon için yolda asal sayılarla ilgili kısıtlamayı kullanmadık. "Asal bir CityId'den gelmedikçe her 10. adım %10 daha uzundur" diyor. Bu nedenle, asal sayıların her 10. adımın başında bitmesini sağlayarak algoritmamızı kurguluyoruz.

Yol boyunca döngü yapıyoruz, ne zaman asal CityId'ye sahip bir şehirle karşılaşırsak, toplam yolu küçültürse, onu 9 ile biten bir dizine sahip (yani 10. adım) yakındaki şehirle değiştirmeye çalışacağız.

Örneğin 76. sırada bir asal şehrimiz varsa, bunun 59., 69., 79. ve 89. sıradaki şehirlerden herhangi biriyle değiştirilip değiştirilemeyeceğini görmeye çalışacağız ve daha kısa bir yol ile sonuçlanacaktır.

Asal sayılı şehri şehri, mevcut şehirden önce gelen ve endeksi 9 ile biten iki şehirle veya mevcut şehirden sonra gelen ve endeksi 9 ile biten iki şehirle değiştirmeye çalışıyoruz.

Şehirleri değiştirerek yolun kısalıp kısalmadığını kontrol ederken, sadece takasa karşılık gelen alt yolun uzunluğunu kontrol edeceğiz. Tüm yolun uzunluğunu kontrol etmeye gerek yok. Bu, aramayı çok daha verimli hale getirecektir.

In [ ]:
nnpath_with_primes = nnpath.copy()
for index in range(20,len(nnpath_with_primes)-30):
    city = nnpath_with_primes[index]
    if (prime_cities[city] &  ((index+1) % 10 != 0)):        
        for i in range(-1,3):
            tmp_path = nnpath_with_primes.copy()
            swap_index = (int((index+1)/10) + i)*10 - 1
            tmp_path[swap_index],tmp_path[index] = tmp_path[index],tmp_path[swap_index]
            if total_distance(df_cities,tmp_path[min(swap_index,index) - 1 : max(swap_index,index) + 2]) < total_distance(df_cities,nnpath_with_primes[min(swap_index,index) - 1 : max(swap_index,index) + 2]):
                nnpath_with_primes = tmp_path.copy() 
                break
print('Total distance with the Nearest Neighbor With Prime Swaps '+  "is {:,}".format(total_distance(df_cities,nnpath_with_primes)))

Birincil takaslarla toplam mesafeyi 650 birim daha azalttık.

### #### En yakın komşu algiritması ile çözülmüş veri setini oluşturalım.


In [ ]:
pd.DataFrame({'Path':nnpath_with_primes}).to_csv('M.Kutlu_SENGUL_TSP3_NN_Prime.csv',index  = False)